# Create simualted datasets

For both the training and testing datasets.

The processing steps include:
- Domain randomisation (relrod, spot_spread)
- Multiple phases
- Corruption of data (disabled for this example)
- Adding nose (S&P)

In [1]:
# Packages
%matplotlib qt
import numpy as np
import hyperspy.api as hs
import pyxem as pxm
import diffpy.structure
from matplotlib import pyplot as plt
from tempfile import TemporaryFile
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator, VectorLibraryGenerator
from pyxem.utils.sim_utils import sim_as_signal
import tqdm
import gc
import os

In [35]:
### Variables

# Paths
root = r'C:\Users\Sauron\Documents\jf631\SED_scripts'
structures_path = os.path.join(root, 'nn_models/crystal_phases')
phase_files = ['cubic_fapbi_scaled.cif', 'pbi2.cif', 'pbbr2.cif', 'pb.cif', 'gratia_4h.cif', 'gratia_6h.cif']

# Calibration values
calibration = 0.0046

# Domain amplification
simulated_direct_beam_bool = [False,]
relrod_list = [0.002, 0.02,]
spot_spread_list = [0.01, 0.02]

# Simulation microscope values
detector_size = 515 #px
beam_energy = 200.0 #keV
wavelength = 2.5079e-12 #m
detector_pix_size = 55e-6 #m
from pyxem.detectors import Medipix515x515Detector
detector = Medipix515x515Detector()

# Processing values
n_angle_points = 2
#corrupt_n_times = 2

cropping_start = 0.11
cropping_stop = 1.30
sqrt_signal = False

## Simulate data for each phase

In [36]:
phase_dict = {}
for phase in phase_files:
    name = phase.split(".")[0]
    phase_dict[name] = diffpy.structure.loadStructure(os.path.join(structures_path, phase))

print('n_phases = {}'.format(len(phase_dict)))

n_phases = 6


In [37]:
def get_random_euler(npoints):
    radius = 1
    np.random.seed(1)
    u = np.random.randint(-100,100+1,size=(npoints,))/100 
    u2 = 2*np.pi*np.random.random(size=(npoints,))
    theta = 2*np.pi*np.random.random(size=(npoints,))
    x = radius*np.sqrt(1-u**2)*np.cos(theta)
    y = radius*np.sqrt(1-u**2)*np.sin(theta)
    z = radius*u 
    phi = np.arccos(z/radius)
    eulerAlpha = u2
    eulerBeta = phi
    eulerGamma = theta
    return np.array([np.rad2deg(eulerAlpha),np.rad2deg(eulerBeta),np.rad2deg(eulerGamma)]).T 


def get_reciprocal_radius(detector_size, calibration):
    half_pattern_size = detector_size // 2
    reciprocal_radius = calibration * half_pattern_size
    return reciprocal_radius


def create_diffraction_library(phase_dict, euler_list,
                                       beam_energy, relrod_length,
                                       calibration, detector_size,
                                       with_direct_beam):

    phase_names = list(phase_dict.keys())
    phases = list(phase_dict.values())
    euler_list_n = [euler_list, ] * len(phase_names)

    sample_lib = StructureLibrary(phase_names, phases, euler_list_n)
    ediff = DiffractionGenerator(beam_energy, relrod_length)
    diff_gen = DiffractionLibraryGenerator(ediff)

    reciprocal_radius = get_reciprocal_radius(detector_size, calibration)
    library = diff_gen.get_diffraction_library(sample_lib,
                                               calibration=calibration,
                                               reciprocal_radius=reciprocal_radius,
                                               half_shape=(detector_size//2, detector_size//2),
                                               with_direct_beam=with_direct_beam)
    return library

In [38]:
data = {}
for key, val in phase_dict.items():
    data[key] = []
for with_direct_beam in simulated_direct_beam_bool:
    for relrod_length in tqdm.tqdm(relrod_list):
        for spot_spread in spot_spread_list:

            euler_list = get_random_euler(n_angle_points)

            library = create_diffraction_library(phase_dict, euler_list,
                                                 beam_energy, relrod_length,
                                                 calibration, detector_size,
                                                 with_direct_beam)

            reciprocal_radius = get_reciprocal_radius(detector_size, calibration)
            for euler in euler_list:
                for phase in library.keys():
                    pattern = sim_as_signal(library.get_library_entry(phase=phase,
                                                                      angle=euler)['Sim'],
                                            detector_size, spot_spread, reciprocal_radius)

                    data[phase].append(pattern)

  0%|          | 0/2 [00:00<?, ?it/s]
                                     
  0%|          | 0/2 [00:00<?, ?it/s]
                                     
  0%|          | 0/2 [00:00<?, ?it/s]
                                     
  0%|          | 0/2 [00:00<?, ?it/s]
                                     
  0%|          | 0/2 [00:00<?, ?it/s]
                                     
100%|██████████| 2/2 [00:00<00:00, 16.42it/s]
                                             
  0%|          | 0/2 [00:00<?, ?it/s]
                                     
  0%|          | 0/2 [00:00<?, ?it/s]
                                     
  0%|          | 0/2 [00:00<?, ?it/s]
                                     
  0%|          | 0/2 [00:00<?, ?it/s]
                                     
  0%|          | 0/2 [00:00<?, ?it/s]
                                     
  0%|          | 0/2 [00:00<?, ?it/s]
                                     
  0%|          | 0/2 [00:00<?, ?it/s]
                                  

This class changed in v0.3 and no longer takes a maximum_excitation_error
This class changed in v0.3 and no longer takes a maximum_excitation_error
This class changed in v0.3 and no longer takes a maximum_excitation_error
This class changed in v0.3 and no longer takes a maximum_excitation_error


In [39]:
# Stack data
import dask.array as da

for i, value in enumerate(data.values()):
    list_data = da.from_array([x.data for x in value], chunks=(10, detector_size, detector_size))

    if i ==0:
        #list_data = np.expand_dims(list_data, 1)
        training_data = list_data
    else:
        #list_data = np.expand_dims(list_data, 1)
        training_data = da.vstack([training_data, list_data],)

del data
del library
del list_data
gc.collect()

shape = (len(phase_dict.keys()),
         n_angle_points*len(relrod_list)*len(spot_spread_list)*len(simulated_direct_beam_bool),
         detector_size,
         detector_size)

training_data = training_data.reshape(shape)
training_data = pxm.LazyElectronDiffraction2D(training_data)
training_data.set_diffraction_calibration(calibration)
print(training_data)

<LazyElectronDiffraction2D, title: , dimensions: (8, 6|515, 515)>


## Recenter

In [40]:
shiftList = np.zeros((np.size(training_data.data,0),
                      np.size(training_data.data,1),
                      2,)
                     )

shiftList[:,:,0]=0.5
shiftList[:,:,1]=0.5

shiftList = shiftList.reshape(-1, shiftList.shape[-1]) # Flatten the 2D navigtion axis

training_data.compute()
training_data.align2D(shifts=shiftList,crop=False,fill_value=0., parallel=True)

name = '2D_simulated_data_{}classes_{}neuler_domainrand_centered_{}cal.hspy'.format(np.size(training_data.data,0),  n_angle_points, calibration)
#training_data.save(os.path.join('2d_simulated_data', name))
print(training_data)

[########################################] | 100% Completed |  0.1s
<ElectronDiffraction2D, title: , dimensions: (8, 6|515, 515)>


C:\Users\Sauron\anaconda3\envs\pyxem\lib\site-packages\dask\array\core.py:1435: FutureWarning: The `numpy.size` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
C:\Users\Sauron\anaconda3\envs\pyxem\lib\site-packages\dask\array\core.py:1435: FutureWarning: The `numpy.size` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,


In [41]:
training_data.data.shape

(6, 8, 515, 515)

In [42]:
training_data.plot()

## Add noise

In two steps:
- S&P noise
- Poisson noise

In [9]:
def add_noise_to_simulation(simulation_arr, snr, int_salt, pdf_scaling_factor,):

    from scipy.stats import multivariate_normal
    import numpy as np

    # Functions
    def random_choice(p):
        return np.random.choice((0, 1, 2), p=[p, (1 - p) / 2., (1 - p) / 2.])

    # To speed up
    random_choice_vec = np.vectorize(random_choice)

    # Salt and pepper
    def addsalt_pepper(im, snr, pdf_scale_fact,  pdf_func = 'gaussian', int_min = 0, int_max = 1, quantile_n=0.99):

        # Create detector pixels matrix
        xlen, ylen = im.shape
        x, y = np.mgrid[0:xlen:1, 0:ylen:1]
        pos = np.dstack((x, y))
        det_center = (int(xlen/2), int(ylen/2))

        # Create probability shape function for the random choise probability accross the 2D detector
        if pdf_func == 'gaussian':
            # Create Gaussian pdf from detector size
            covar = np.zeros((2, 2))
            np.fill_diagonal(covar, xlen * 3) # is HARD CODED VALUE FOR DETECTOR (TO MATCH EXP DATA)
            rv = multivariate_normal(det_center, covar)
            pdf = rv.pdf(pos)

        if pdf_func == 'inv_quadratic':
            # Create 1/q^2 pdf from detector size
            def f_inv_q2(mesh, quantile_n):
                mesh[mesh == 0] = 1e-1
                x = mesh[:,:,0]
                y = mesh[:,:,1]
                # Get the q^2 ratio
                z = 1 / (x**2 + y**2)
                # Simulate detector saturation
                quantile = np.quantile(z, q=quantile_n)
                z[z > quantile] = quantile
                z = z / z.max()
                return z

            pos_centre = (pos - det_center)
            pdf = f_inv_q2(pos_centre, quantile_n)


        # Normalise to snr so max val is snr
        pdf *= snr / pdf.max()
        # Create weighted probability array for choice 0 (no change)
        p0 = np.ones(im.shape) * snr
        # Subtract the effect the probabiliy of being off-centered following pdf function defined above
        p0 -= pdf * pdf_scale_fact

        # Add noise
        mask = random_choice_vec(p0)

        im[mask == 1] = int_min # salt noise
        im[mask == 2] = int_max # pepper noise

        return im

    # Create saltpepper noise basis
    im = simulation_arr
    im_sp = addsalt_pepper(im, snr, pdf_scale_fact= pdf_scaling_factor, int_max=int_salt)

    # Add poisson noise on sp noise
    im = im_sp + np.random.poisson(im)

    return im

In [10]:
# Create iteration tools
from itertools import product
import numpy as np

# Variables
sim_numpy_array = training_data.data
sim_numpy_array = sim_numpy_array.reshape((sim_numpy_array.shape[0]*sim_numpy_array.shape[1], sim_numpy_array.shape[2], sim_numpy_array.shape[3]))

snr_saltpepper = [0.99, 0.9999]
intensity_salt = np.linspace(0.02, 0.1, 2)
pdf_scaling_factor = np.linspace(0, 0.3, 2)


iterations = product(sim_numpy_array, snr_saltpepper,
                     intensity_salt, pdf_scaling_factor,)


In [11]:
# make sure to always use multiprocess
from multiprocess import Pool
import psutil

# start your parallel workers at the beginning of your script
n_cores = psutil.cpu_count(logical=False)
pool = Pool(n_cores)

# execute a computation(s) in parallel
im_stack = pool.starmap(add_noise_to_simulation, iterations)

# turn off your parallel workers at the end of your script
pool.close()

In [12]:
np.shape(im_stack)

(12800, 515, 515)

In [22]:
how_many = 10
f, ax = plt.subplots(ncols=how_many)
for i,n in enumerate(np.random.randint(low=0, high=np.shape(im_stack)[0], size=how_many)):
    ax[i].imshow(im_stack[n], vmax=0.1)

## Integrate radially

In [14]:
# Define function

def integration_in_parallel_from_array(array_im, calibration, return_hspy_object=False):
    """
    array_im: single image array
    """
    # import_packages
    import pyxem as pxm
    import gc
    import numpy as np
    from pyxem.detectors import Medipix515x515Detector

    detector = Medipix515x515Detector()
    dp = pxm.signals.electron_diffraction2d.ElectronDiffraction2D(array_im)

    radial_int_points = 256
    acceleration_v = 200
    wavelength = 2.5079e-12
    pix_size = 55e-6 #change to 1 if using the GenericFlatDetector()

    camera_length = pix_size / (wavelength * calibration * 1e10)
    dp.beam_energy = acceleration_v
    dp.set_diffraction_calibration(calibration)
    dp.unit = 'k_A^-1'
    dp.set_experimental_parameters(beam_energy=acceleration_v)

    radial = dp.get_azimuthal_integral1d(npt_rad=radial_int_points,
                                         center=([dp.axes_manager.signal_shape[0]/2-1, dp.axes_manager.signal_shape[1]/2-1]),
                                         detector=detector,
                                         detector_dist=camera_length)

    del dp
    gc.collect()

    if return_hspy_object:
        return radial
    else:
        return radial.data

In [15]:
# Define parameters

calibration = [0.0046, ]

# Create iteration tools
from itertools import product

iterations = product(im_stack, calibration)

In [16]:
# Run
# make sure to always use multiprocess
from multiprocess import Pool
import psutil

# start your parallel workers at the beginning of your script
n_cores = psutil.cpu_count(logical=False)
pool = Pool(n_cores)

# execute a computation(s) in parallel
im_stack_radial = pool.starmap(integration_in_parallel_from_array, iterations)

# turn off your parallel workers at the end of your script
pool.close()

In [17]:
np.shape(im_stack_radial)

(12800, 256)

## Corrupt data several times

In [66]:
def create_random_dampening_profile(signal_array):
    sig_len = len(signal_array)
    dumb = np.repeat(np.random.choice([0,1,1],38),(sig_len//50))
    dumb1 = np.append(dumb,np.zeros([sig_len-len(dumb),]))
    dumbrnd = np.repeat(np.random.rand(15,),sig_len//15)
    dumbrnd1 = np.append(dumbrnd,np.zeros([sig_len-len(dumbrnd),]))
    dempening_profile = dumb1 * dumbrnd1
    return dempening_profile

def dampen_signal(signal_array):
    dampening_profile = create_random_dampening_profile(signal_array)
    return signal_array * dampening_profile


In [67]:
training_data_1D_corrupted = training_data_1D.data

for i in range(corrupt_n_times):
    damped = training_data_1D.map(dampen_signal, inplace=False, parallel=True)
    training_data_1D_corrupted = np.append(training_data_1D_corrupted, damped, axis=1)

## Crop and normalise and sqrt

In [68]:
training_data_1D_corrupted = pxm.ElectronDiffraction1D(training_data_1D_corrupted,)
training_data_1D_corrupted.axes_manager.signal_axes[0].scale = training_data_1D.axes_manager.signal_axes[0].scale
training_data_1D_corrupted.axes_manager.signal_axes[0].offset = training_data_1D.axes_manager.signal_axes[0].offset


training_data_1D_corrupted.crop_signal1D(cropping_start, cropping_stop)

if sqrt_signal:
    training_data_1D_corrupted.data = np.sqrt(training_data_1D_corrupted.data)

dpmax = training_data_1D_corrupted.data.max(2)
training_data_1D_norm = training_data_1D_corrupted.data/dpmax[:,:,np.newaxis]

print(training_data_1D_norm.shape)

(4, 9000, 182)


C:\Users\Sauron\anaconda3\envs\pyxem\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


## NN requirements: reshape and labelling

In [69]:
training_data_1D_norm = training_data_1D_norm.reshape(-1, training_data_1D_norm.shape[-1])

print(training_data_1D_norm.shape)

(36000, 182)


In [71]:
# Create labels
n_phases = len(phase_dict)
labels = np.zeros((n_phases, int(training_data_1D_norm.shape[0]/n_phases)))
for i in range(n_phases):
    labels[i,:] = i

training_labels = labels.flatten()

In [ ]:
# Check for outliers and nan values
where_nan = np.argwhere(np.isnan(training_data_1D_norm))
training_data_1D_norm = np.delete(training_data_1D_norm, where_nan[:,0], axis = 0)
training_labels = np.delete(training_labels, where_nan[:,0], axis = 0)
print(training_data_1D_norm.shape, training_labels.shape)

In [72]:
store_train_data = TemporaryFile()
x = training_data_1D_norm
y = training_labels
phase_names = list(phase_dict.keys())

In [73]:
np.savez('1D_simulated_data_{}classes_{}neuler_domainrand_{}ncorrupted'.format(n_phases,  n_angle_points, corrupt_n_times), x=x, y=y, phases=phase_names)

In [1]:
print(training_data_1D_corrupted.data.shape)


NameError: name 'training_data_1D_corrupted' is not defined